This notebook continuously estimates the prevailing, realtime, positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>

## Preliminaries

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.investigations.prospects
import atlantic.investigations.hospitalizations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

<br>

**Attributes**

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 
         'hospitalizedIncrease': np.float64, 'positiveCumulative': np.float64, 'testCumulative': np.float64, 
         'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 'positiveRate': np.float64, 
         'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>
<br>

**The URL strings of the sources**

In [14]:
uribaseline = os.path.join(parent, 'warehouse', 'baselines.csv')
urihospitalizations = os.path.join(parent, 'warehouse', 'hospitalizations.csv')

<br>
<br>

**Baseline**

In [15]:
baselines = pd.read_csv(filepath_or_buffer=uribaseline, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)
baselines.loc[:, 'deathPositiveRate'] = np.where(baselines['positiveRate'] > 0, 100 * baselines['deathRate'] / baselines['positiveRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16328 entries, 0 to 16327
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          16328 non-null  datetime64[ns]
 1   STUSPS                  16328 non-null  object        
 2   deathIncrease           16328 non-null  float64       
 3   deathCumulative         16328 non-null  float64       
 4   positiveIncrease        16328 non-null  float64       
 5   positiveCumulative      16328 non-null  float64       
 6   hospitalizedIncrease    16328 non-null  float64       
 7   hospitalizedCumulative  16328 non-null  float64       
 8   testIncrease            16328 non-null  float64       
 9   testCumulative          16328 non-null  float64       
 10  deathRate               16328 non-null  float64       
 11  positiveRate            16328 non-null  float64       
 12  testRate                16328 non-null  float6

<br>
<br>

**Hospitalisations**

In [16]:
hospitalizations = pd.read_csv(filepath_or_buffer=urihospitalizations, header=0, 
                               usecols=fields,dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

hospitalizations.loc[:, 'hospitalizedPositiveRate'] =  np.where(hospitalizations['positiveRate'] > 0, 
                                                                100 * hospitalizations['hospitalizedRate'] / hospitalizations['positiveRate'], 0)
hospitalizations.loc[:, 'deathHospitalizedRate'] =  np.where(hospitalizations['hospitalizedRate'] > 0, 
                                                             100 * hospitalizations['deathRate'] / hospitalizations['hospitalizedRate'], 0)

logger.info(hospitalizations.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11618 entries, 0 to 11617
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            11618 non-null  datetime64[ns]
 1   STUSPS                    11618 non-null  object        
 2   deathIncrease             11618 non-null  float64       
 3   deathCumulative           11618 non-null  float64       
 4   positiveIncrease          11618 non-null  float64       
 5   positiveCumulative        11618 non-null  float64       
 6   hospitalizedIncrease      11618 non-null  float64       
 7   hospitalizedCumulative    11618 non-null  float64       
 8   testIncrease              11618 non-null  float64       
 9   testCumulative            11618 non-null  float64       
 10  deathRate                 11618 non-null  float64       
 11  positiveRate              11618 non-null  float64       
 12  testRate          

INFO:__main__:None


<br>
<br>

## Regression

<br>

**Latest Date**

In [17]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2020-11-30 00:00:00


<br>
<br>

**Periods**

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [18]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

**Partitions**

In [19]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### **positives vs. tests**

<br>

**Analysis**

In [20]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [21]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of affairs**

In [22]:
ptr = estimates

<br>

Prevailing Positive Test Rate

In [23]:
ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [24]:
ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'positiveTestRate': 'continuousPTR'},
          inplace=True)
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')
ptr.to_json(path_or_buf=os.path.join(warehouse, 'ptr.json'), orient='values')

ptr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingPTR,continuousPTR,rank,prevailingTPC
6,2020-11-30,0.0523064,0.0452365,0.0548014,2.62745e-22,0.997803,-1468.91,CT,5.23064,3.596402,48.0,19.1181
45,2020-11-30,0.0514505,0.0358699,0.061697,1.70706e-14,0.976154,-1180.7,VT,5.14505,1.860142,49.0,19.4362
32,2020-11-30,0.0314348,0.0287179,0.0391866,1.38085e-20,0.995966,168.456,NY,3.14348,3.336406,50.0,31.8119
8,2020-11-30,0.0232647,0.0191168,0.0263505,1.79806e-16,0.98832,758.799,DC,2.32647,3.099865,51.0,42.9836
19,2020-11-30,0.0224731,0.00797058,0.0364888,1.206e-13,0.970297,-578.166,ME,2.24731,1.342431,52.0,44.4977


<br>
<br>

### **deaths vs. positives**

In [25]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [26]:
formula = 'deathRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=20, num=5, latestdate=latestdate)

In [27]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of Affairs**

In [28]:
dpr = estimates

<br>

Prevailing Death Positive Rate

In [29]:
dpr.loc[:, 'prevailingDPR'] = 100 * dpr['gradient']
dpr = dpr.merge(baselines[['datetimeobject', 'STUSPS', 'deathPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
dpr.loc[:, 'cf'] = dpr[['prevailingDPR', 'deathPositiveRate']].max(axis=1)

dpr.loc[:, 'rank'] = dpr['cf'].rank(method='min', ascending=False)
dpr.sort_values(by='rank', ascending=True, inplace=True)
dpr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [30]:
dpr.loc[:, 'prevailingCPD'] = np.where(dpr['prevailingDPR'] > 0, dpr['prevailingDPR'].rdiv(100), 0)
dpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'deathPositiveRate': 'continuousDPR'},
          inplace=True)
dpr.to_csv(path_or_buf=os.path.join(warehouse, 'dpr.csv'), index=False, header=True, encoding='utf-8')
dpr.to_json(path_or_buf=os.path.join(warehouse, 'dpr.json'), orient='values')

dpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDPR,continuousDPR,rank,prevailingCPD
34,2020-11-30,0.00679418,-0.000687519,0.0096477,1.21999e-07,0.83411,21.8097,ND,0.679418,0.873164,48.0,147.185
27,2020-11-30,0.00739278,0.00402893,0.00965604,2.14231e-14,0.976059,3.3857,NE,0.739278,0.782028,49.0,135.267
50,2020-11-30,0.00741315,0.000505007,0.0108875,1.46661e-12,0.959435,-3.95965,WY,0.741315,0.645549,50.0,134.895
44,2020-11-30,0.00363166,0.00312655,0.00558784,7.1275e-16,0.986149,4.92125,UT,0.363166,0.445055,51.0,275.356
1,2020-11-30,0.00296377,0.000836387,0.00578225,8.06131e-09,0.881761,4.00452,AK,0.296377,0.386298,52.0,337.408


<br>
<br>

### **hospitalizations vs. positives**

In [31]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [32]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [33]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [34]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [35]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [36]:
hpr = hpr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')
hpr.to_json(path_or_buf=os.path.join(warehouse, 'hpr.json'), orient='values')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
32,2020-11-30,0.0159874,0.00758979,0.0210439,1.49428e-13,0.969492,90.6233,TN,1.59874,33.0,3.229967,62.5491
33,2020-11-30,0.0139678,0.00782246,0.0353874,3.43132e-08,0.858214,158.861,MA,1.39678,34.0,6.252543,71.593
34,2020-11-30,0.00522233,0.00286204,0.00675519,1.90517e-11,0.953336,54.0753,NH,0.522233,35.0,4.096680,191.485
35,2020-11-30,5.55112e-17,-5.08652e-15,5.19755e-15,0.623872,-50.3077,343.787,CT,5.55112e-15,36.0,10.449721,1.80144e+16
36,2020-11-30,1.38778e-16,-9.19075e-15,9.85689e-15,0.750475,-40.9615,462.615,NY,1.38778e-14,36.0,13.888546,7.20576e+15


<br>
<br>

### **deaths vs. hospitalizations**

In [37]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [38]:
formula = 'deathRate ~ hospitalizedRate'
regressor = 'hospitalizedRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [39]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [40]:
dhr = estimates

<br>

Prevailing

In [41]:
dhr.loc[:, 'prevailingDHR'] = 100 * dhr['gradient']
dhr.loc[:, 'rank'] = dhr['prevailingDHR'].rank(method='min', ascending=False)
dhr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [42]:
dhr = dhr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'deathHospitalizedRate']], how='left', on=['datetimeobject', 'STUSPS'])
dhr.loc[:, 'prevailingHPD'] = np.where(dhr['prevailingDHR'] > 0, dhr['prevailingDHR'].rdiv(100), 0)

dhr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'deathHospitalizedRate': 'continuousDHR'}, 
           inplace=True)

dhr.to_csv(path_or_buf=os.path.join(warehouse, 'dhr.csv'), index=False, header=True, encoding='utf-8')
dhr.to_json(path_or_buf=os.path.join(warehouse, 'dhr.json'), orient='values')

dhr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDHR,rank,continuousDHR,prevailingHPD
32,2020-11-30,0.132201,0.103712,0.173993,6.37178e-17,0.986922,44.5868,AZ,13.2201,33.0,25.746529,7.56421
33,2020-11-30,0.131737,0.0106243,0.167436,3.09573e-10,0.920981,6.51771,AL,13.1737,34.0,14.121083,7.59087
34,2020-11-30,0.120722,0.085141,0.144219,7.90347e-18,0.988378,11.246,VA,12.0722,35.0,15.709479,8.2835
35,2020-11-30,0.114127,0.0962508,0.168655,5.3868e-16,0.986098,-1.89077,UT,11.4127,36.0,10.706822,8.7622
36,2020-11-30,0.112915,0.0884007,0.118362,6.75883e-21,0.99631,9.08471,OK,11.2915,37.0,14.219285,8.85619


<br>
<br>

## Clean-up

In [43]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states